<a href="https://colab.research.google.com/github/Danicoder/Especializacion_IA_BigData/blob/main/Clasificador_flores_mejorado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from keras import Sequential
from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Descargar y extraer el dataset
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, cache_dir='.', untar=True) #untar=True creó una subcarpeta adicional llamada flower_photos
data_dir = os.path.join(os.path.dirname(data_dir), 'flower_photos')  # Obtener ruta correcta
test_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, cache_dir='.', untar=True)
test_dir = os.path.join(os.path.dirname(test_dir), 'flower_photos_test')

In [ ]:
print(test_dir)

./datasets/flower_photos_test


In [ ]:
# Verificar si la carpeta "flower_photos" existe dentro del directorio de datos
flower_photos_dir = os.path.join(data_dir, "flower_photos")

# Actualizar data_dir para que apunte a flower_photos si existe
if os.path.exists(flower_photos_dir):
    data_dir = flower_photos_dir  # Actualizamos a la ruta correcta
    print("Ruta actualizada a:", data_dir)
else:
    print("La carpeta 'flower_photos' no existe en el directorio de datos.")

Ruta actualizada a: ./datasets/flower_photos/flower_photos


In [ ]:
# Verificar si la carpeta "flower_photos" existe dentro del directorio de datos
flower_photos_test = os.path.join(test_dir, "flower_photos")

# Actualizar data_dir para que apunte a flower_photos si existe
if os.path.exists(flower_photos_test):
    test_dir = flower_photos_test  # Actualizamos a la ruta correcta
    print("Ruta actualizada a:", test_dir)
else:
    print("La carpeta 'flower_photos' no existe en el directorio de datos.")

La carpeta 'flower_photos' no existe en el directorio de datos.


In [ ]:
# Parámetros del dataset
batch_size = 32
img_height = 160
img_width = 160

# Configurar ImageDataGenerator para cargar imágenes por lotes
# ImageDataGenerator para entrenamiento y validación (con división de 80% y 20%)
train_datagen = ImageDataGenerator(rescale=1.0/255., validation_split=0.2)
# ImageDataGenerator para test (con división de 20%)
test_datagen = ImageDataGenerator(rescale=1.0/255.)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training' # División para entrenamiento (80%)
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation' # División para validación (20%)
)
# Para el conjunto de test, no usamos subset ni validation_split
test_generator =  test_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
    subset='validation'
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-10-4c944d4a9e94>, line 32)

#ENTRENAMIENTO DEL MODELO Y AJUSTE

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Obtener un lote de imágenes del generador
images, labels = next(train_generator)

# Visualizar las primeras 5 imágenes
plt.figure(figsize=(10, 10))
for i in range(5):
    ax = plt.subplot(1, 5, i + 1)
    plt.imshow(images[i])  # Muestra la imagen
    plt.axis("off")  # No mostrar los ejes
plt.show()


Data augmentation

In [ ]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal", # Volteo aleatorio
        input_shape=(img_height,
                     img_width,
                     3)),
    layers.RandomRotation(0.2), # Rotación aleatoria (hasta 288°)
    layers.RandomZoom(0.2),
    layers.RandomBrightness(0.5)                   # Ajuste de brillo aleatorio (mas o menos 20%)
  ])

Arquitectura del modelo con MobileNetV2

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(160, 160, 3),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
vgg16_conv_base = VGG16(
    input_shape = (
        img_height,img_width,3),  # Tamaño de la imágen de entrada con 160 canales * 3 (rojo,verde y azul)
        include_top=False,        # excluye la capa final
        weights = 'imagenet')     # conjunto de datos

vgg16_conv_base.summary()

Capas

Entrenar modelo

In [ ]:
model = Sequential()
model.add(base_model)
model.add(data_augmentation)

# evito sobreajuste
model.add(Dropout(0.2))
# Aplanar las características
model.add(Flatten())

# Capa densa con 128 neuronas
model.add(Dense(128, activation='relu'))
# Capa de salida con 5 neuronas
model.add(Dense(5, activation='softmax'))

model.summary()

Compilar modelo

In [ ]:
model.compile(
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
      train_generator,
      epochs=10,
      validation_data=validation_generator,
      verbose=1,
      callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6),
        tf.keras.callbacks.ModelCheckpoint(filepath='modelo_flowers.h5', monitor='val_loss', save_best_only=True)
      ]
)

Evaluo el modelo

In [ ]:
test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'  # Change class_mode to 'sparse'
)


Visualizar las Curvas de accuracy y val_accuracy

In [ ]:
# Graficar la precisión y la precisión de validación
plt.figure(figsize=(8, 6))

# Accuracy de entrenamiento
plt.plot(history.history['accuracy'], label='Training accuracy')

# Accuracy de validación
plt.plot(history.history['val_accuracy'], label='Validation accuracy')

# Añadir etiquetas
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Mostrar la gráfica
plt.show()

Transfer Learning (Aumenta la presición del la red)

Fine-Tuning: Descongelar Capas del Modelo Base

In [ ]:
vgg16_conv_base.trainable = True # no entrenar capa convulcional de VGG16

# Define TrainingConfig as a simple class to hold configuration
class TrainingConfig:
    LAYERS_FINE_TUNE = 8

num_layers_fine_tune = TrainingConfig.LAYERS_FINE_TUNE
num_layers = len(vgg16_conv_base.layers)

# Congelamos las capas que no vamos a entrenar
for layer in vgg16_conv_base.layers[:num_layers - num_layers_fine_tune]:
    print(f"FREEZING LAYER: {layer}")
    layer.trainable = False

print("\n")
print(f"Configured to fine tune the last {num_layers_fine_tune} layers of the base model.")
print("\n")

# Verificamos el estado de las capas
print(vgg16_conv_base.summary())

